In [1]:
%pylab inline
import sys; sys.path.insert(0, "../")
import numpy as np
import timeit
from emcee.autocorr import *
from ensemble_sampler import *

Populating the interactive namespace from numpy and matplotlib


### Example of sampling from an invariant distribution of SPDE  
Ref. Goodman & Weare, Ensemble Samplers With Affine Invariance  
A problem with this example is that it's harder to visualize, but from the experiments it does seem that pCN has more consistent performance in high dimensions (judging from acceptance probability).

In [2]:
# Fixed for all experiments in this notebook
nwalkers = 100
batch_size = 50

In [3]:
def plot_traj(x):
    n, dim = x.shape
    for i in range(n):
        t = np.linspace(start=0.0, stop=1.0, num=dim)
        plt.plot(t, x[i, :])

In [4]:
def run(dim, proposal, batch_size=50, niters=1000, n=5, pre=0):
    t_dist = SPDE(N=dim)
    sampler = Sampler(dim=dim, t_dist=t_dist, proposal=proposal, nwalkers=nwalkers)

    acc_r = 0.0
    for i in range(n):
        sampler.reset()
        p0 = np.random.randn(dim*nwalkers).reshape([nwalkers, dim])
        if pre > 0:
            hist = sampler.run_mcmc(pre, batch_size=batch_size, p0=p0, verbose=False)
            p0 = hist.curr_pos
            sampler.reset()
        start = timeit.default_timer()
        hist = sampler.run_mcmc(niters, batch_size=batch_size, p0=p0, verbose=False)
        print 'finishes loop %d in %.2f seconds' % (i, float(timeit.default_timer() - start))
        acc_r += float(100*hist.acceptance_rate.mean())
        # print 'avg accept rate: %.2f%s' % (float(100*hist.acceptance_rate.mean()), '%')
        try:
            print 'mean auto-correlation time: %s' % hist.auto_corr().mean()
        except AutocorrError, err:
            pass
    print 'avg_acc_r: %.2f%s' % (float(acc_r) / 5.0, '%')

#### Not using ensemble, use isotropic gaussian proposal with scale=0.001

In [5]:
dim = 50
proposal = PCNWalkMove(s=None, scale=0.001)

run(dim, proposal, batch_size=batch_size, niters=10000, n=5)

finishes loop 0 in 5.80 seconds
finishes loop 1 in 5.82 seconds
finishes loop 2 in 5.54 seconds
finishes loop 3 in 5.28 seconds
finishes loop 4 in 5.29 seconds
avg_acc_r: 81.83%


#### Use ensemble of size 3, scale=0.005.  
Experimented with dim=50 & 100, see a decrease in acceptance rate with same scaling parameter.

In [6]:
dim = 50
proposal = PCNWalkMove(s=3, scale=0.005)

run(dim, proposal, batch_size=batch_size, niters=10000, n=5)

finishes loop 0 in 6.38 seconds
finishes loop 1 in 5.69 seconds
finishes loop 2 in 5.66 seconds
finishes loop 3 in 5.66 seconds
finishes loop 4 in 5.71 seconds
avg_acc_r: 83.05%


In [7]:
dim = 100
proposal = PCNWalkMove(s=3, scale=0.005)

run(dim, proposal, batch_size=batch_size, niters=10000, n=5)

finishes loop 0 in 6.95 seconds
finishes loop 1 in 7.47 seconds
finishes loop 2 in 7.01 seconds
finishes loop 3 in 7.47 seconds
finishes loop 4 in 7.06 seconds
avg_acc_r: 69.90%


In [8]:
dim = 200
proposal = PCNWalkMove(s=3, scale=0.005)

run(dim, proposal, batch_size=batch_size, niters=10000, n=5)

finishes loop 0 in 9.95 seconds
finishes loop 1 in 9.99 seconds
finishes loop 2 in 9.94 seconds
finishes loop 3 in 10.03 seconds
finishes loop 4 in 10.00 seconds
avg_acc_r: 48.51%


#### Ensemble with pCN, beta=0.2  
Experimented with dim=50 & 100, consistency in acceptance rate with same scaling parameter.

In [9]:
dim = 50
proposal = PCNWalkMove(s=3, beta=0.2)

run(dim, proposal, batch_size=batch_size, niters=10000, n=5)

finishes loop 0 in 5.96 seconds
finishes loop 1 in 5.84 seconds
finishes loop 2 in 5.76 seconds
finishes loop 3 in 5.77 seconds
finishes loop 4 in 5.80 seconds
avg_acc_r: 85.88%


In [10]:
dim = 100
proposal = PCNWalkMove(s=3, beta=0.2)

run(dim, proposal, batch_size=batch_size, niters=10000, n=5)

finishes loop 0 in 7.15 seconds
finishes loop 1 in 7.09 seconds
finishes loop 2 in 7.13 seconds
finishes loop 3 in 7.14 seconds
finishes loop 4 in 7.11 seconds
avg_acc_r: 85.34%


In [11]:
dim = 200
proposal = PCNWalkMove(s=3, beta=0.2)

run(dim, proposal, batch_size=batch_size, niters=10000, n=5)

finishes loop 0 in 10.20 seconds
finishes loop 1 in 10.17 seconds
finishes loop 2 in 10.20 seconds
finishes loop 3 in 10.20 seconds
finishes loop 4 in 10.24 seconds
avg_acc_r: 84.87%


#### PCN without ensemble, beta=0.001  
Need small beta to get good acceptance rate.

In [ ]:
dim = 50
proposal = PCNWalkMove(s=None, beta=0.001)

run(dim, proposal, batch_size=batch_size, niters=30000, n=5)

finishes loop 0 in 17.62 seconds
finishes loop 1 in 18.39 seconds


In [ ]:
dim = 100
proposal = PCNWalkMove(s=None, beta=0.001)

run(dim, proposal, batch_size=batch_size, niters=30000, n=5)

In [ ]:
dim = 200
proposal = PCNWalkMove(s=None, beta=0.001)

run(dim, proposal, batch_size=batch_size, niters=30000, n=5)